# PlaiCraft DSCI 100 Individual Planning

In this individual planning, I will be analyzing and visualizing data provided by Frank Wood's Computer Science research group for their vanilla survival MineCraft server, PlaiCraft, to prepare for the group aspect of the project where we will answer an agreed upon predictive question. The datasets provided were players.csv and sessions.csv.

### Data Description and Inspection

Before we start analyzing, here are a few details about the collection of the data, concluded from the datasets and PlaiCraft website:
 - From the sessions dataset, data was collected between May 1 - September 1, 2024
 - Study runs under approval from UBC's office of research ethics
 - Players can halt their data being collected if they'd like
 - Hashed emails collected is where PlaiCraft access links were sent
 - Those 13 years old and under required guardian consent to collect their data
 - Data for each session was collected from when game browser opened to closed
 - Each server was monitored consistently
 - Sessions range from solo to multiplayer; but multiplayer was encouraged

After previewing both datasets, there looks to be two NA values in players.csv in rows 159 and 197 and missing data in row 682 and 1020 of sessions.csv. One option could be to drop these rows, but there may be valuable information in the other variables of those rows, so I will decide to keep them and keep note of them for future summarizing and wrangling.

So, let's begin! First, let's load in the tidyverse, ggplot2, RColorBrewer, and lubridate R packages to use for future analysis.

In [ ]:
library(tidyverse)
library(ggplot2)
library(lubridate)
library(RColorBrewer)
set.seed(9999)

### 1. Inspecting the Datasets with R Functions and Summaries

Let's read in the two datasets and inspect what data we have to work with.

In [ ]:
players_data <- read_csv("data/players.csv")
players_data

#### Description of the set:

The "players.csv" has 7 variables and 196 observations, as listed above the data table (196x7). The variables, their data type, and description of what they represent are listed below:
1. `experience`: *chr (character)* - determines the "level" of the players' gaming experience as a Beginner, Amateur, Regular, Veteran, or Pro (the most experienced)
2. `subscribe`: *lgl (logical)* - indicated with TRUE for "yes" or FALSE for "no", determining whether or not the player is subscribed to PlaiCraft's newsletter
3. `hashedEmail`: *chr (character)* - email address of player, but in hashed form with a sequence of letters and numbers
4. `played_hours`: *dbl (double (decimal))* - how many hours the player played PlaiCraft for
5. `name`: *chr (character)* - first name of player
6. `gender`: *chr (character)* - gender of player
7. `age`: *dbl (double (decimal))* - age of player


One issue seen is the subscribe column. I assume it's based on subscription for the game's newsletter, as mentioned in one of the broad questions. However, there is no metadata to verify this. Likewise, I'm assuming the two datasets were collected during the same timespan as they were provided together. There is also no indication in the players.csv dataset about how long the data collection was for though. Furthermore, the order of the experience column is unclear. Usually the order from least to most experience is the one listed above. But, there is also no indication about it. As well, there was no information about how one could be classified as each experience category and whether the player themselves, categorized themself, in which may bring up bias.

#### Summary #1

In [ ]:
experience_count <- players_data |>
                        group_by(experience) |>
                        summarize(count = n())

experience_summary <- experience_count |>
                        ungroup() |>
                        mutate(percent = count/sum(count) * 100) |>
                        mutate(percent = round(percent, 2))
experience_summary

##### Experience Summary:
- Amateur (largest portion) = 63 = 32.14%
- Beginner = 35 = 17.86%
- Pro (smallest portion) = 14 = 7.14%
- Regular = 36 = 18.37%
- Veteran = 48 = 24.49%

#### Summary #2

In [ ]:
played_hours_summary <- players_data |>
                            summarize(mean = mean(played_hours, na.rm = TRUE),
                                      median = median(played_hours, na.rm = TRUE),
                                      sum = sum(played_hours, na.rm = TRUE),
                                      max = max(played_hours),
                                      min = min(played_hours)) |>
                            mutate(across(mean:min, ~ round(.x, 2)))    
played_hours_summary                      

##### Played Hours Summary:
- Mean = 5.85 hours
- Median = 0.1 hours
- Sum = 1145.80 hours
- Max = 223.10 hours
- Min = 0 hours

#### Summary #3

In [ ]:
subscribe_count <- players_data |>
                        group_by(subscribe) |>
                        summarize(count = n())

subscribe_summary <- subscribe_count |>
                        ungroup() |>
                        mutate(percent = count/sum(count) * 100) |>
                        mutate(percent = round(percent, 2))
subscribe_summary

##### Subscribe Summary:
- FALSE = 52 = 26.53%
- TRUE = 144 = 73.47%

#### Summary #4

In [ ]:
gender_count <- players_data |>
                    group_by(gender) |>
                    summarize(count = n())

gender_summary <- gender_count |>
                        ungroup() |>
                        mutate(percent = count/sum(count) * 100) |>
                        mutate(percent = round(percent, 2))
gender_summary

##### Gender Summary:
- Agender = 2 = 1.02%
- Female = 37 = 18.88%
- Male (largest portion) = 124 = 63.27%
- Non-binary = 15 = 7.65%
- Other (smallest portion) = 1 = 0.51%
- Prefer not to say = 11 = 5.61%
- Two-spirited = 6 = 3.06%

#### Summary #5

In [ ]:
age_summary <- players_data |>
                    summarize(mean = mean(Age, na.rm = TRUE),
                              median = median(Age, na.rm = TRUE),
                              max = max(Age, na.rm = TRUE),
                              min = min(Age, na.rm = TRUE)) |>
                    mutate(across(mean:min, ~ round(.x, 2)))    
age_summary   

##### Age Summary:
- Mean = 21.14 years old
- Median = 19 years old
- Oldest (max) = 58 years old
- Youngest (min) = 9 years old.

**Now, let's read in the sessions dataset!**

In [ ]:
sessions_data <- read_csv("data/sessions.csv")
sessions_data

#### Description of the set:
Indicated at the top of the data table, "sessions.csv" has 5 variables and 1535 observations (1535x5). The variables, their data type, and description of what they represent are listed below: 
1. `hashedEmail`: *chr (character)* - the players' email, but in hashed form
2. `start_time`: *chr (character)* - start time of players' gametime session in "mm/dd/yyyy" and "time (in 24 hour interval)"
4. `end_time`: *chr (character)* - end time of players' gametime session in "mm/dd/yyyy" and "time (in 24 hour interval)"
5. `original_start_time`: *dbl (double (decimal))* - players' start time in UNIX format (milliseconds)
6. `original_end_time`: *dbl (double (decimal))* - players' end time in UNIX format (milliseconds)


One issue seen in the data is the formatting of the start_time, end_time, original_start_time, and original_end_time columns. There is more than one value in the start_time and end_time which is not favourable as a tidy dataset and for applying functions to the dataset later. The original_start_time and original_end_time columns are also in an uncommonly used format and difficult to interpet.
\
\
Before we summarize, I will wrangle the data to make it tidy. Tidy data follows the format of one variable per column, one observation per row, and no more than one value in each cell. The "players.csv" follows the tidy rules, so we didn't have to tidy it. I will separate the values in the start_time and end_time columns so that it's one value per cell. I will transform those values into a decimal type instead of character. I will add an extra column for the start_time in hours as integers for later analysis. All this wrangling will make summarizing and creating visualizations later more efficient.

In [ ]:
sessions_data_tidy <- sessions_data |>
                      separate(end_time, 
                                into = c("date", "end_time"),
                                sep = " ") |>
                      separate(start_time,
                                into = c("date", "start_time"),
                                sep = " ") |>
                       mutate(start_time = as.numeric(hm(start_time))/3600) |> #hm function is from the lubridate R package and extracts the time in seconds
                       mutate(end_time = as.numeric(hm(end_time))/3600) |> #we devide it by 3600 to get the time in hours
                       mutate(start_time_hr = as.integer(start_time)) |>
                       mutate(start_time_hr = start_time_hr %% 24) #%% will divide each value by 24 and output the remainder; we want this so that any 24 hour values = 0 on the 24 hour interval
sessions_data_tidy

#### Summary #6

In [ ]:
date_total <- sessions_data_tidy |>
                    group_by(date) |>
                    summarize(count = n()) |>
                    arrange(desc(count))

pull(head(date_total, 1))
pull(tail(date_total, 1))

We can assume that the highest number of sessions in one day was 38 and lowest was 1. Let's filter to see if there are any more days with these same counts.

In [ ]:
date_summary <- date_total |>
                    filter(count %in% c(38, 1))

date_summary

##### Sessions in a Day Summary:
- Least active days were in July and September, with one in April and June as well = 1 session 
- Most popular day of playing was July 25, 2024 = 38 sessions

#### Summary #7

In [ ]:
start_time_summary <- sessions_data_tidy |>
                            summarize(mean = mean(start_time, na.rm = TRUE),
                                      median = median(start_time, na.rm = TRUE),
                                      max = max(start_time, na.rm = TRUE),
                                      min = min(start_time, na.rm = TRUE)) |>
                            mutate(across(mean:min, ~ round(.x, 2)))
start_time_summary

##### Start Time Summary:
- Mean = 10.69 hours = 10:41am
- Median = 6.53 hours = 6:31am
- Latest (max) = 23.98 hours = 11:58pm
- Earliest (min) = 0 hours = 12:00am

#### Summary #8

In [ ]:
end_time_summary <- sessions_data_tidy |>
                            summarize(mean = mean(end_time, na.rm = TRUE),
                                      median = median(end_time, na.rm = TRUE),
                                      max = max(end_time, na.rm = TRUE),
                                      min = min(end_time, na.rm = TRUE)) |>
                            mutate(across(mean:min, ~ round(.x, 2)))
end_time_summary

##### End Time Summary:
- Mean = 10.09 hours = 10:05am
- Median = 6.25 hours = 6:15am
- Latest (max) = 23.98 hours = 11:58pm
- Earliest (min) = 0 hours = 12:00am

### 2. The Question
Now that we have taken a view at our data, let's state the question.

The broad question being addressed: "We are interested in demand forecasting, namely, what time windows are most likely to have large number of simultaneous players. This is because we need to ensure that the number of licenses on hand is sufficiently large to accommodate all parallel players with high probability." 

With this, the specific predictive question I formulated is: 
##### Can players' Plaicraft session start time predict how many simultaneous players will be online so that we can determine if we have enough licenses to go out at any time of the day?

### 3. Exploratory Data Analysis and Visualization


Next, let's create some exploratory visualizations to understand the data more and possibly seek out any helpful relationships. Visualizations make it much easier to interpret datasets and make large datasets into concise figures.

Some of the graphs provide information that is not as important for my specific question, but it is worth while to create these visualizations still to further understand and explore both datasets as a whole. It may be needed for when I come together with my group as well.

#### Visualization #1

In [ ]:
options(repr.plot.width = 11, repr.plot.height = 9)
played_hours_age_plot <- ggplot(players_data, aes(x = Age, y = played_hours + 1)) + #add +1 so that when we log our y axis, the 0 values won't be infinity
                            geom_point(alpha = 0.5) +
                            labs(x = "Age (Years)", y = "Hours of PlaiCraft Played (Scaled)") +
                            ggtitle("Age of PlaiCraft Players vs Hours Played") +
                            scale_y_log10() +
                            theme(text = element_text(size = 12))
played_hours_age_plot

There doesn't seem to be any relationship between the age of the players and the hours of PlaiCraft played as the points are widespread and there is no clear trend. The points are pretty condensed near the bottom of the graph, with some outliers near the top half. The graph seems to insinuate that teens and young adults tend to play for greater amounts of hours, specifically ages around 18 - 20. This is not only seen by the outliers, but nearby the condensed area as well.

#### Visualization #2

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 6)
age_histogram <- ggplot(players_data, aes(x = Age)) +
                    geom_histogram(bins = 20) +
                    labs(x = "Age (Years)", y = "Total Players") +
                    ggtitle("Distribution of PlaiCraft's Players' Ages") +
                    scale_x_continuous(breaks = c(0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65)) +
                    scale_y_continuous(breaks = c(0, 10, 20, 30, 40, 50, 60, 70, 80, 90)) +
                    theme(text = element_text(size = 15))
age_histogram

In this visualization, we can see that the majority of PlaiCraft players in the dataset are approximately 17 years old. A lot of the players' ages are condensed around the teenager to young adult ages of 16 - 25. There are however, a few beyond this at the young children, adult and senior ages.

#### Visualization #3

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 6)
start_time_players_total <- group_by(sessions_data_tidy, start_time_hr) |>
                                summarize(total_players = n())

start_time_players_line <- ggplot(start_time_players_total, aes(x = start_time_hr, y = total_players)) +
                                geom_line(alpha= 0.6) +
                                scale_x_continuous(breaks = seq(0, 23)) +
                                labs(x = "Hour of Day", y = "Total Players Online") +
                                ggtitle("Total Players Online at Each Hour of the Day") +
                                theme(text = element_text(size = 15))
start_time_players_line

This visualization is the most helpful for our specific question. It's just a rough estimate of how many players are online at certain hours since I converted the time of day values to integers. So, it may not be the exact amount of players, but it should be relatively close. We can see that from 2 - 5am, a large amount of players are online. The number of players online is the lowest between 8am-2pm, and increases again from 3pm to the end of the day. I can connect these observations to visualization #2 in that I can assume that young adults are most likely in class or working. It makes sense why visualization #2 increases from 3pm to the end of the day all the way to 5am the next day because a lot of workers and students who are young adults get their free time late at night.

#### Visualization #4

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 6)

experience_age_histogram <- age_histogram +
                                ggtitle("Distribution of Ages Across Different Gaming Experiences") +
                                scale_y_continuous(breaks = c(0,2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28)) +
                                scale_x_continuous(breaks = c(0, 10, 20, 30, 40, 50, 60)) +
                                facet_grid(cols = vars(experience))

experience_age_histogram

This is an extension of visualization #2, split by gaming experience. We can conclude that age does not correlate to gaming experience as in, just because a player is on the fairly younger side, it does not mean that they are automatically a beginner or amateur. They could be as equally a pro as 20-30 year olds as shown in the pro column. This also goes the same for more elderly people. Although many seem to be catergorized as amateur, there are definitely a few in veteran. However, that being said, as stated before, there's no explanation of how this category was collected. So, there could be some bias.

#### Visualization #5

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 6)

gender_bar <- ggplot(players_data, aes(x = gender, fill = subscribe)) +
                geom_bar(position = "stack") +
                labs(x = "Gender", y = "Count", fill = "Are they Subscribed?") +
                ggtitle("PlaiCraft Players by Gender") +
                scale_fill_brewer(palette = "Set1") +
                scale_y_continuous(breaks = c(0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130)) +
                theme(text = element_text(size = 15))
gender_bar

We can see that the majority of PlaiCraft players identify as male, with female coming in second, and non-binary coming in third. Overall, for the majority of the gender categories, more than half of each count of players are subscribed to PlaiCraft's newsletter. 

#### Visualization #6

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 5)
experience_subscribe_bar <- ggplot(players_data, aes(x = experience, fill = subscribe)) +
                                geom_bar(position = "fill") +
                                labs(x = "Gaming Experience", y = "Ratio of Players", fill = "Are they Subscribed?") +
                                ggtitle("Subscription of Players Based on Gaming Experience") +
                                scale_fill_brewer(palette = "Set2") +
                                theme(text= element_text(size = 14))
experience_subscribe_bar

It seems like regular players are most likely to subscribe to PlaiCraft's newsletter compared to the other gaming experiences. Although, across all gaming experiences, there are similar proportions of subscribed and not subscribed.

### 4. Methods and Plans

The method I would use to address my question of interest is non-linear regression. I would work on the sessions.csv dataset as it contains the variables and data that will help address the specific question. I can do the same mutating of the time values in the "start_time" column of the sessions.csv dataset shown earlier and group them in intervals of for example, 15 minutes; this will be the explanatory variable. Additionally, I would summarize the data to make a column based on how many players were starting at each of those intervals; this will be the response variable. I will then do the necessary training and validation of the data to create an optimal knn non-linear regression model. My question is more dependent on time of sessions and how many players are playing. Thus, the players.csv dataset does not contain any variables that will effectively aid my analysis.

The model chosen is appropriate because I am looking to predict a numerical value, which compared to classification, would not be able to do. Also, with basing my classifer off of the explanatory variable, "start_time", and response variable, total players online, I would be able to efficiently input any time of the day and based on *k*, the classifier will output a prediction of total players online at that time. I can then decide myself if the number of players online is exceeding the number of licenses I currently have at hand for that time of day and decide what the best course of action is for the licenses.

Fortunately, the non-linear regression model does not require much assumptions compared to the linaar one where it assumes that the data is linear. A reason for its minimal assumptions is due to its nearest neighbours system. However, with minimal assumptions, comes setbacks.

One setback is that it doesn't predict well when the new observation is outside the range of the data it was trained with. Moreover, as the dataset gets larger, the longer it takes to compute the output. Fortunately, this dataset is not too large so far, so it will not take long to compute an average outcome. However, it can become a problem for when more data is added to get a more accurate outcome. 

##### *To process the data, I will follow the usual steps for creating a non-linear knn regression model. I will list the usual steps and make note of specific changes below:*
1. Inspect the data and deal with any missing values and problems appropriately
2. Split the dataset into 75% training and 25% testing.
3. Tune the training set
    - Set the mode to "regression" in *nearest_neighbor*
    - In the recipe: the response variable = total_players_online, explanatory variable = start_time
        - No need to scale as there is only one explanatory variable
4. Cross validation on training set
        - Do 5 folds
    - Make a tibble with the range of neighbors using *tibble()*
        - Range will be from 1 to 96 (24hrs = 1440mins, 1440mins/15min interval = 96 data points)
        - Neighbors will differ by 5; ex. neighbors = 1, 6, 11... 96
    - Create a workflow with *workflow()* and use *tune_grid* and *collect_metrics*
5. Filter and slice for the optimal *k*
6. Assess on testing data by making new knn model and workflow, then using *predict* and *bind_cols*, and collecting the metrics with *metrics*
7. Compare RMSPE to cross-validation RMSE
8. Use the optimal *k* and fit into our original dataset to create our final mo

### References

Wickham, Hadley, and Garrett Grolemund. 2016. R for Data Science: Import, Tidy, Transform, Visualize, and Model Data. O’Reilly. https://r4ds.had.co.nz/.
\
\
The Pacific Laboratory of Artificial Intelligence. FAQ. Plaicraft. https://plaicraft.ai/faq. 